In [1]:
import glob
import numpy as np
import pandas as pd
import anndata
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize, OneHotEncoder
from sklearn.decomposition import PCA, TruncatedSVD
from ALLCools.mcds import MCDS
from ALLCools.plot import *
from ALLCools.clustering import *
from wmb import brain, aibs, broad, mm10
import pynndescent
from scipy.sparse import csr_matrix


In [2]:
indir = '/home/jzhou_salk_edu/sky_workdir/230327_Kim_integration/'

In [9]:
file_list = glob.glob(f'{indir}data/10x*/*/')
file_list

['/home/jzhou_salk_edu/sky_workdir/230327_Kim_integration/data/10x_VMH_Male_Aggression_3/2018_0803/',
 '/home/jzhou_salk_edu/sky_workdir/230327_Kim_integration/data/10x_VMH_Male_M-M_CI_dangled_2/2019_0414/',
 '/home/jzhou_salk_edu/sky_workdir/230327_Kim_integration/data/10x_VMH_Male_M-M_CI_dangled_1/2019_0331/',
 '/home/jzhou_salk_edu/sky_workdir/230327_Kim_integration/data/10x_VMH_Female_Control_1/2018_0428/',
 '/home/jzhou_salk_edu/sky_workdir/230327_Kim_integration/data/10x_VMH_Male_Others_4/2019_0413/',
 '/home/jzhou_salk_edu/sky_workdir/230327_Kim_integration/data/10x_VMH_Male_Control_2/2018_0429/',
 '/home/jzhou_salk_edu/sky_workdir/230327_Kim_integration/data/10x_VMH_Male_M-M_CI_pencil_cup_1/2018_0703/',
 '/home/jzhou_salk_edu/sky_workdir/230327_Kim_integration/data/10x_VMH_Female_Mating_not_receptive_2/2018_0812_2/',
 '/home/jzhou_salk_edu/sky_workdir/230327_Kim_integration/data/10x_VMH_Male_Plain_2/2018_0527/',
 '/home/jzhou_salk_edu/sky_workdir/230327_Kim_integration/data/10x

In [10]:
for file in file_list[18:]:
    data = sc.read_10x_mtx(file)
    data.write_h5ad(f'/home/jzhou_salk_edu/sky_workdir/230327_Kim_integration/data/{file.split("/")[-3]}.h5ad')
    print(file)
    

/home/jzhou_salk_edu/sky_workdir/230327_Kim_integration/data/10x_VMH_Male_Social_Fear_Singly-housed_1/2018_0812_1/
/home/jzhou_salk_edu/sky_workdir/230327_Kim_integration/data/10x_VMH_Male_Others_2/2019_0323/
/home/jzhou_salk_edu/sky_workdir/230327_Kim_integration/data/10x_VMH_Male_Control_3/2018_0520/
/home/jzhou_salk_edu/sky_workdir/230327_Kim_integration/data/10x_VMH_Male_M-F_CI_dangled_1/2018_0519/
/home/jzhou_salk_edu/sky_workdir/230327_Kim_integration/data/10x_VMH_Male_Control_1/2018_0420/
/home/jzhou_salk_edu/sky_workdir/230327_Kim_integration/data/10x_VMH_Female_Mating_not_receptive_1/2018_0802/
/home/jzhou_salk_edu/sky_workdir/230327_Kim_integration/data/10x_VMH_Female_Plain_1/2018_0707/
/home/jzhou_salk_edu/sky_workdir/230327_Kim_integration/data/10x_VMH_Male_Mating_1/2018_0715/


In [11]:
print(len(file_list))

26


In [3]:
meta = pd.read_csv(f'{indir}data/10x_metadata_cell.csv', header=0, index_col=0)
meta

,orig.ident,umis_label,genes_label,behavior_label,sex_label,sex_exp_label,housing_label,tech,percent.mt,cell_cluster_id,...,cell_cluster_color,neuron_cluster_id,neuron_cluster_label,neuron_cluster_color,VMH_neuron_cluster_id,VMH_neuron_cluster_label,VMH_neuron_cluster_color,VMH_neuron_cca_cluster_id,VMH_neuron_cca_cluster_label,VMH_neuron_cca_cluster_color
sample_name,,,,,,,,,,,,,,,,,,,,,
1_AAACCTGAGCTGATAA,10x_VMH_Male_Plain_1,1499,949,Plain,M,Naive,Group,10X_v2,5.603736,27.0,...,#4D4535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1_AAACCTGCACTGTCGG,10x_VMH_Male_Plain_1,3312,1923,Plain,M,Naive,Group,10X_v2,6.129227,20.0,...,#763BFF,15.0,Dlk1_4,#4D8092,15.0,Dlk1_2,#74DCFF,16.0,Dlk1_2,#66ABC2
1_AAACCTGCATAGTAAG,10x_VMH_Male_Plain_1,1430,935,Plain,M,Naive,Group,10X_v2,7.062937,28.0,...,#141211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1_AAACCTGCATTCCTCG,10x_VMH_Male_Plain_1,1788,1148,Plain,M,Naive,Group,10X_v2,3.803132,55.0,...,#9C9C9C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1_AAACCTGGTAGCGATG,10x_VMH_Male_Plain_1,2736,1365,Plain,M,Naive,Group,10X_v2,6.652047,24.0,...,#89ADA4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26_TTTGTCACAAGTCATC,10x_VMH_Male_M-M_CI_dangled_2,2648,1341,CI(M)_dangle,M,Exp,Single,10X_v2,7.703927,24.0,...,#89ADA4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26_TTTGTCACAGACGCTC,10x_VMH_Male_M-M_CI_dangled_2,5023,2712,CI(M)_dangle,M,Exp,Single,10X_v2,4.618754,5.0,...,#C129A4,24.0,Gluta_VMH-out_1,#CC4E3D,NaN,NaN,NaN,NaN,NaN,NaN
26_TTTGTCACAGTATAAG,10x_VMH_Male_M-M_CI_dangled_2,3158,1934,CI(M)_dangle,M,Exp,Single,10X_v2,4.844839,25.0,...,#354D40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
sample_map = meta[['orig.ident']]
sample_map['sample'] = sample_map.index.str.split('_').str[0]
sample_map = sample_map.drop_duplicates().set_index('orig.ident')['sample']
sample_map

orig.ident
10x_VMH_Male_Plain_1                         1
10x_VMH_Male_Control_1                       2
10x_VMH_Male_Aggression_1                    3
10x_VMH_Female_Control_1                     4
10x_VMH_Male_Control_2                       5
10x_VMH_Male_M-F_CI_dangled_1                6
10x_VMH_Male_Control_3                       7
10x_VMH_Male_Plain_2                         8
10x_VMH_Male_Aggression_2                    9
10x_VMH_Male_M-F_CI_pencil_cup_1            10
10x_VMH_Male_M-M_CI_pencil_cup_1            11
10x_VMH_Female_Plain_1                      12
10x_VMH_Male_Mating_1                       13
10x_VMH_Male_Others_1                       14
10x_VMH_Female_Mating_not_receptive_1       15
10x_VMH_Male_Aggression_3                   16
10x_VMH_Male_Social_Fear_Singly-housed_1    17
10x_VMH_Female_Mating_not_receptive_2       18
10x_VMH_Female_Mating_not_receptive_3       19
10x_VMH_Male_Others_2                       20
10x_VMH_Male_Others_3                       21
10

In [5]:
meta.columns

Index(['orig.ident', 'umis_label', 'genes_label', 'behavior_label',
       'sex_label', 'sex_exp_label', 'housing_label', 'tech', 'percent.mt',
       'cell_cluster_id', 'cell_cluster_label', 'cell_cluster_color',
       'neuron_cluster_id', 'neuron_cluster_label', 'neuron_cluster_color',
       'VMH_neuron_cluster_id', 'VMH_neuron_cluster_label',
       'VMH_neuron_cluster_color', 'VMH_neuron_cca_cluster_id',
       'VMH_neuron_cca_cluster_label', 'VMH_neuron_cca_cluster_color'],
      dtype='object')

In [6]:
file_list = glob.glob(f'{indir}data/*.h5ad')
print(len(file_list))


26


In [15]:
data = []
for file in file_list:
    xx = file.split('/')[-1].split('.')[0]
    tmp = anndata.read_h5ad(file)
    tmp.obs.index = str(sample_map[xx]) + '_' + tmp.obs.index.str.split('-').str[0]
    tmp = tmp[tmp.obs.index.isin(meta.index)]
    tmp.var['n_cells'] = tmp.X.getnnz(axis=0)
    tmp = tmp[:, tmp.var['n_cells']>(0.001*tmp.shape[0])]
    data.append(tmp)
    print(tmp.shape)
    

(5496, 18251)
(7952, 16292)
(5174, 16962)
(9994, 17033)
(7025, 17761)
(5906, 18294)
(6414, 16001)
(5971, 18054)
(7089, 17488)
(8569, 16207)
(5032, 17832)
(4688, 16208)
(5953, 17861)
(6306, 17106)
(6221, 17885)
(6642, 17718)
(7318, 17644)
(6767, 18125)
(7500, 17306)
(8342, 17306)
(6198, 17792)
(2727, 16002)
(7910, 17427)
(9681, 17253)
(3100, 17917)
(4902, 17970)


In [16]:
data = anndata.AnnData.concatenate(*data, index_unique=None)
data.obs = meta.loc[data.obs.index].copy()
data

AnnData object with n_obs × n_vars = 168877 × 15014
    obs: 'orig.ident', 'umis_label', 'genes_label', 'behavior_label', 'sex_label', 'sex_exp_label', 'housing_label', 'tech', 'percent.mt', 'cell_cluster_id', 'cell_cluster_label', 'cell_cluster_color', 'neuron_cluster_id', 'neuron_cluster_label', 'neuron_cluster_color', 'VMH_neuron_cluster_id', 'VMH_neuron_cluster_label', 'VMH_neuron_cluster_color', 'VMH_neuron_cca_cluster_id', 'VMH_neuron_cca_cluster_label', 'VMH_neuron_cca_cluster_color'
    var: 'gene_ids', 'n_cells-0', 'n_cells-1', 'n_cells-10', 'n_cells-11', 'n_cells-12', 'n_cells-13', 'n_cells-14', 'n_cells-15', 'n_cells-16', 'n_cells-17', 'n_cells-18', 'n_cells-19', 'n_cells-2', 'n_cells-20', 'n_cells-21', 'n_cells-22', 'n_cells-23', 'n_cells-24', 'n_cells-25', 'n_cells-3', 'n_cells-4', 'n_cells-5', 'n_cells-6', 'n_cells-7', 'n_cells-8', 'n_cells-9'

In [17]:
data.var = data.var[['gene_ids']]

In [19]:
data.X.data = data.X.data/np.repeat(data.obs['umis_label'].values, data.X.getnnz(axis=1)
                                     ) * data.obs['umis_label'].median()
sc.pp.log1p(data)

In [21]:
remove_gene = pd.read_csv(f'{indir}data/removed_gene.tsv', header=0, index_col=None, sep='\t')
remove_gene

,Sex-specific,IEGs,retro-virus-induced,noise-sensitive
0,Ddx3y,Fos,B2m,Nrgn
1,Eif2s3y,Fosb,Bst2,Tmsb10
2,Uty,Fosl1,Oasl2,Basp1
3,Kdm5d,Fosl2,Ifit1,Rpl9-ps6
4,Xist,Jun,H2-D1,Gng3
...,...,...,...,...
995,NaN,NaN,NaN,Car10
996,NaN,NaN,NaN,Gabrb3
997,NaN,NaN,NaN,Cpe
998,NaN,NaN,NaN,Grik2


In [24]:
selg = remove_gene['IEGs']
selg = selg.loc[~selg.isna()]
selg

0         Fos
1        Fosb
2       Fosl1
3       Fosl2
4         Jun
        ...  
134     Gdf15
135      Ier5
136      Rgs1
137       Id2
138    Apold1
Name: IEGs, Length: 139, dtype: object

In [28]:
selg = data.var.index[data.var.index.isin(selg)]
tmp = data[:, selg].X.toarray()
tmp = pd.DataFrame(tmp, columns=selg, index=data.obs.index)


In [34]:
data.obsm['IEGs'] = tmp.copy()

In [40]:
selg = remove_gene.values.flatten()
selg = selg[selg.astype(str)!='nan']
print(len(selg))

1175


In [43]:
data = data[:, ~data.var.index.isin(selg)].copy()
data = data[data.obs['neuron_cluster_label'].astype(str)!='nan'].copy()
data

AnnData object with n_obs × n_vars = 78476 × 13889
    obs: 'orig.ident', 'umis_label', 'genes_label', 'behavior_label', 'sex_label', 'sex_exp_label', 'housing_label', 'tech', 'percent.mt', 'cell_cluster_id', 'cell_cluster_label', 'cell_cluster_color', 'neuron_cluster_id', 'neuron_cluster_label', 'neuron_cluster_color', 'VMH_neuron_cluster_id', 'VMH_neuron_cluster_label', 'VMH_neuron_cluster_color', 'VMH_neuron_cca_cluster_id', 'VMH_neuron_cca_cluster_label', 'VMH_neuron_cca_cluster_color'
    var: 'gene_ids'
    uns: 'log1p'
    obsm: 'IEGs'

In [44]:
data.write_h5ad('act.h5ad')

In [3]:
data = anndata.read_h5ad('act.h5ad')
data

AnnData object with n_obs × n_vars = 78476 × 5314
    obs: 'orig.ident', 'umis_label', 'genes_label', 'behavior_label', 'sex_label', 'sex_exp_label', 'housing_label', 'tech', 'percent.mt', 'cell_cluster_id', 'cell_cluster_label', 'cell_cluster_color', 'neuron_cluster_id', 'neuron_cluster_label', 'neuron_cluster_color', 'VMH_neuron_cluster_id', 'VMH_neuron_cluster_label', 'VMH_neuron_cluster_color', 'VMH_neuron_cca_cluster_id', 'VMH_neuron_cca_cluster_label', 'VMH_neuron_cca_cluster_color', 'Study', 'leiden', 'leiden_score'
    var: 'gene_ids'
    uns: 'log1p'
    obsm: 'IEGs', 'X_pca', 'X_pca_corrected', 'cef_pca'

In [11]:
data.obs['VMH_neuron_cluster_label'].astype(str)=='nan'

13_AAACCTGGTCGCTTCT     True
13_AAACCTGTCCAAATGC    False
13_AAACCTGTCGCGATCG    False
13_AAACGGGCACTGCCAG    False
13_AAACGGGCAGGGAGAG     True
                       ...  
26_TTTGGTTTCCGTAGTA    False
26_TTTGGTTTCGATCCCT     True
26_TTTGTCACAGACGCTC     True
26_TTTGTCAGTGTGGTTT     True
26_TTTGTCATCTGATTCT    False
Name: VMH_neuron_cluster_label, Length: 78476, dtype: bool